In [97]:
import itertools

import pandas as pd
import numpy as np

import linearmodels

In [137]:
susb = (
    pd.read_csv('data/final/susb_combined.csv')
    .set_index(['initial_year', 'naics_code', 'var'])
    ['value']
    .unstack()
    .reset_index()
    .rename(columns={'initial_year': 'year'})
    .assign(empl_created=lambda df: df['empl_births'] + df['empl_expand'])
    .assign(empl_destroyed=lambda df: 0 - df['empl_deaths'] - df['empl_contract'])
)

dhs = pd.DataFrame({
    'year': susb['year'] + 1,
    'naics_code': susb['naics_code'],
    'estb_dhs': susb['estb_initial'] + (susb['estb_births'] - susb['estb_deaths']) / 2,
    'empl_dhs': susb['empl_initial'] + (susb['empl_created'] - susb['empl_destroyed']) / 2,
})

susb = (
    susb
    .merge(dhs, how='outer')
    .assign(estb_birth_rate=lambda df: 100 * df['estb_births'] / df['estb_dhs'])
    .assign(empl_create_rate=lambda df: 100 * df['empl_created'] / df['empl_dhs'])
    .assign(empl_destroy_rate=lambda df: 100 * df['empl_destroyed'] / df['empl_dhs'])
)
    

In [99]:
rd31_3digit = (
    pd.read_csv('data/rd31/cfr_naics07-3digit_extract.csv')
    .rename(columns={'industry-relevant restrictions': 'regdata31'})
    [['year', 'naics_code', 'regdata31']]
)

In [4]:
rd31_4digit = (
    pd.read_csv('data/rd31/cfr_naics07-4digit_extract.csv')
    .rename(columns={'industry-relevant restrictions': 'regdata31'})
    [['year', 'naics_code', 'regdata31']]
)

In [5]:
rd20_3digit = (
    pd.read_csv('data/rd20/three_digit_industry_RegData2_0.csv')
    .groupby(['year', 'industry_code'])['industry_regulation_index'].sum()
    .reset_index()
    .rename(columns={
        'industry_code': 'naics_code',
        'industry_regulation_index': 'regdata20'
    })
    .loc[lambda df: df['regdata20'] > 0]
)

In [7]:
rd21_3digit = (
    pd.read_csv(
        'data/rd21/industry_classification_probabilities.csv',       
        index_col=['year', 'title', 'part']
    )
    .mul(
        (
            pd.read_csv(
                'data/rd21/cfrref.csv',
                index_col=['year', 'title', 'part'],
                encoding='latin-1'
            )
            ['restrictions']
        ),
        axis=0
    )
    .groupby(level='year').sum()
    .reset_index()
    .melt(id_vars='year', var_name='naics_code', value_name='regdata21')
    .assign(naics_code=lambda df: df['naics_code'].astype(int))
)

In [8]:
rd22_3digit = (
    pd.read_csv(
        'data/rd22/RegData_2_2_by_3-digit_industry.csv',
        names=['year', 'naics_code', 'restrictions', 'words'],
        skiprows=1,
    )
    .rename(columns={'restrictions': 'regdata22'})
    [['year', 'naics_code', 'regdata22']]
)

In [9]:
rd22_4digit = (
    pd.read_csv(
        'data/rd22/RegData_2_2_by_4-digit_industry.csv',
        names=['year', 'naics_code', 'restrictions', 'words'],
        skiprows=1
    )
    .rename(columns={'restrictions': 'regdata22'})
    [['year', 'naics_code', 'regdata22']]
)

In [139]:
combined_3digit = (
    susb[
        (susb['naics_code'] >= 100)
        & (susb['naics_code'] < 814)
    ]
    .assign(estb_births=lambda df: df[df['estb_births'] > 0]['estb_births'])
    .assign(estb_deaths=lambda df: df[df['estb_deaths'] > 0]['estb_deaths'])
    .assign(empl_created=lambda df: df[df['empl_created'] > 0]['empl_created'])
    .merge(rd20_3digit, how='left')
    .merge(rd21_3digit, how='left')
    .merge(rd22_3digit, how='left')
    .merge(rd31_3digit, how='left')
)
combined_4digit = (
    susb[
        (susb['naics_code'] >= 1000)
        & (susb['naics_code'] < 8140)
    ]
    .assign(estb_births=lambda df: df[df['estb_births'] > 0]['estb_births'])
    .assign(estb_deaths=lambda df: df[df['estb_deaths'] > 0]['estb_deaths'])
    .assign(empl_created=lambda df: df[df['empl_created'] > 0]['empl_created'])
    .merge(rd20_4digit, how='left')
    .merge(rd22_4digit, how='left')
    .merge(rd31_4digit, how='left')
)

In [140]:
summary_table_rows = {
    'estb_birth_rate': 'Startup Rate',
    'empl_create_rate': 'Job Creation Rate',
    'empl_destroy_rate': 'Job Destruction Rate',
    'estb_births': 'Establishment Births',
    'estb_deaths': 'Establishment Deaths',
    'empl_created': 'New Hires',
    'regdata20': 'RegData 2.0 Regulation Index',
    'regdata21': 'RegData 2.1 Regulation Index',
    'regdata22': 'RegData 2.2 Regulation Index',
    'regdata31': 'RegData 3.1 Regulation Index'
}

(
    combined_3digit
    .loc[lambda df: df['year'] <= 2010]
    .describe()
    [[i for i in summary_table_rows if i in combined_3digit]]
    .round(2)
    .rename(columns=summary_table_rows)
    .T
    [['count', 'mean', 'std', 'min', 'max']]
    .rename(columns={'std':'standard deviation'})
    .assign(count=lambda df: df['count'].astype(int))
    .applymap(lambda x: f'{x:,.0f}' if x > 100 else f'{x:.02f}')
)

,count,mean,standard deviation,min,max
Startup Rate,"1,030",10.84,4.83,1.05,46.15
Job Creation Rate,970,14.58,6.65,1.35,81.29
Job Destruction Rate,969,15.11,5.33,0.94,61.84
Establishment Births,"1,118","8,422","14,666",1.00,"105,010"
Establishment Deaths,"1,116","8,110","13,526",1.00,"94,476"
New Hires,"1,075","209,215","333,911",746,"2,297,342"
RegData 2.0 Regulation Index,869,"5,480","10,007",0.36,"63,506"
RegData 2.1 Regulation Index,975,"32,585","28,725","4,387","143,593"
RegData 2.2 Regulation Index,650,"8,717","13,006",21.12,"66,351"
RegData 3.1 Regulation Index,650,"16,644","20,416",233,"97,502"


In [141]:
(
    combined_4digit.describe()
    [[i for i in summary_table_rows if i in combined_4digit]]
    .round(2)
    .rename(columns=summary_table_rows)
    .T
    [['count', 'mean', 'std', 'min', 'max']]
    .rename(columns={'std':'standard deviation'})
    .assign(count=lambda df: df['count'].astype(int))
    .applymap(lambda x: f'{x:,.0f}' if x > 100 else f'{x:.02f}')
)

,count,mean,standard deviation,min,max
Startup Rate,"4,056",10.58,8.05,0.00,370
Job Creation Rate,"3,658",14.57,6.80,1.35,84.67
Job Destruction Rate,"3,645",14.78,5.75,0.94,81.63
Establishment Births,"4,637","2,482","4,423",1.00,"38,742"
Establishment Deaths,"4,636","2,353","4,090",1.00,"40,944"
New Hires,"4,377","61,888","115,468",162,"1,396,593"
RegData 2.0 Regulation Index,"2,956","1,124","3,935",0.00,"36,131"
RegData 2.2 Regulation Index,"1,632","2,778","3,970",8.61,"25,482"
RegData 3.1 Regulation Index,"2,142","10,226","16,530",149,"84,274"


# Regressions

In [174]:
def get_model(dep, ind, data, logdep=True, cluster=False):
    safe_data = (
        data
        [[dep, ind, 'naics_code', 'year']]
        .dropna()
        .set_index(['naics_code', 'year'])
    )
    if logdep:
        formula = f'np.log({dep}) ~ 1 + np.log({ind}) + EntityEffects + TimeEffects'
    else:
        formula = f'{dep} ~ 1 + np.log({ind})'
    print(formula)
    model = linearmodels.PanelOLS.from_formula(formula, data=safe_data)
    if cluster:
        fitted = model.fit(cov_type='clustered', cluster_entity=True)
    else:
        fitted = model.fit(cov_type='robust')
    print(fitted.summary)
    return (
        fitted.params[1],
        fitted.std_errors[1],
        fitted.pvalues[1]
    )


In [175]:
DIGITS = (
    (3, combined_3digit),
    (4, combined_4digit)
)

INDVARS = (
    'regdata20',
    'regdata21',
    'regdata22',
    'regdata31',
)

DEPVARS = (
    ('estb_births', True), # Variable, Log
    ('estb_deaths', True),
    ('empl_created', True),
    ('estb_birth_rate', False),
    ('empl_create_rate', False),
    ('empl_destroy_rate', False)
)

CLUSTER = (True, False)

In [176]:
results = []
for (digits, data), indvar, (depvar, logdep), cluster in itertools.product(DIGITS, INDVARS, DEPVARS, CLUSTER):
    if not indvar in data:
        continue
    param, stderr, pval = get_model(depvar, indvar, data, logdep, cluster)
    results.append([digits, depvar, indvar, param, stderr, pval, 'clustered' if cluster else 'robust'])
results = pd.DataFrame(results, columns=['digits', 'depvar', 'indvar', 'param', 'stderr', 'pval', 'errtype'])

np.log(estb_births) ~ 1 + np.log(regdata20) + EntityEffects + TimeEffects
                           PanelOLS Estimation Summary                           
Dep. Variable:     np.log(estb_births)   R-squared:                        0.0025
Estimator:                    PanelOLS   R-squared (Between):              0.0040
No. Observations:                  936   R-squared (Within):               0.0065
Date:                 Tue, Jun 11 2019   R-squared (Overall):              0.0030
Time:                         17:13:03   Log-likelihood                    47.611
Cov. Estimator:              Clustered                                           
                                         F-statistic:                      2.1099
Entities:                           67   P-value                           0.1467
Avg Obs:                        13.970   Distribution:                   F(1,855)
Min Obs:                        12.000                                           
Max Obs:                

In [179]:
def format_row(row):
    return (
        f'{row["param"]:01.02f} '
        f'({row["pval"]:01.02f})'
        f'{"*" if row["pval"] < 0.1 else ""}'
        f'{"*" if row["pval"] < 0.05 else ""}'
        f'{"*" if row["pval"] < 0.01 else ""}'
    )

(
    results
    .set_index(['errtype', 'indvar', 'digits', 'depvar'])
    .apply(format_row, axis=1)
    .unstack()
    .sort_index()
    [['estb_births', 'estb_deaths', 'empl_created', 'estb_birth_rate', 'empl_create_rate', 'empl_destroy_rate']]
).xs('clustered')

depvar               estb_births   estb_deaths     empl_created  \
indvar    digits                                                  
regdata20 3         -0.06 (0.47)  -0.08 (0.45)     -0.21 (0.11)   
          4         -0.02 (0.20)  -0.01 (0.68)  -0.06 (0.01)***   
regdata21 3       -0.32 (0.02)**  -0.25 (0.15)  -0.52 (0.00)***   
regdata22 3          0.05 (0.53)   0.07 (0.31)      0.01 (0.90)   
          4        -0.16 (0.06)*  -0.10 (0.17)   -0.11 (0.03)**   
regdata31 3         -0.19 (0.31)  -0.22 (0.25)     -0.27 (0.13)   
          4          0.18 (0.15)   0.11 (0.33)      0.14 (0.32)   

depvar           estb_birth_rate empl_create_rate empl_destroy_rate  
indvar    digits                                                     
regdata20 3          0.21 (0.26)     -0.05 (0.85)      -0.14 (0.50)  
          4          0.13 (0.31)      0.01 (0.92)      -0.13 (0.10)  
regdata21 3         -0.09 (0.86)      0.10 (0.89)      -0.77 (0.21)  
regdata22 3          0.08 (0.79)      0.44 (0.25)       0.07 (0.80)  
          4          0.20 (0.30)      0.34 (0.25)      -0.04 (0.85)  
regdata31 3         -0.34 (0.39)   -1.01 (0.05)**    -0.87 (0.02)**  
          4         -0.37 (0.22)     -0.49 (0.14)    -0.57 (0.02)**

In [178]:
from statsmodels.formula import api as smf

safe_data = combined_3digit[['year', 'naics_code', 'estb_birth_rate', 'regdata20']].dropna()
(
    smf.ols('estb_birth_rate ~ np.log(regdata20) + C(year) + C(naics_code)', safe_data)
    .fit()
    .summary()
)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        estb_birth_rate   R-squared:                       0.730
Model:                            OLS   Adj. R-squared:                  0.703
Method:                 Least Squares   F-statistic:                     26.93
Date:                Tue, 11 Jun 2019   Prob (F-statistic):          1.35e-174
Time:                        17:13:16   Log-Likelihood:                -2056.8
No. Observations:                 868   AIC:                             4274.
Df Residuals:                     788   BIC:                             4655.
Df Model:                          79                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                9.8202      3.122      3.146      0.002       3.692      15.948
C(year)[T.2000]          0.3600      0.475      0.758      0.448      -0.572       1.292
C(year)[T.2001]          0.4108      0.473      0.868      0.385      -0.518       1.339
C(year)[T.2002]         -0.3962      0.473     -0.837      0.403      -1.325       0.532
C(year)[T.2003]          0.4345      0.473      0.918      0.359      -0.495       1.364
C(year)[T.2004]         -0.5132      0.474     -1.083      0.279      -1.443       0.417
C(year)[T.2005]          0.2221      0.475      0.467      0.640      -0.711       1.155
C(year)[T.2006]          1.3652      0.474      2.880      0.004       0.435       2.296
C(year)[T.2007]         -1.5452      0.475     -3.253      0.001      -2.478      -0.613
C(year)[T.2008]         -2.5060      0.476     -5.263      0.000      -3.441      -1.571
C(year)[T.2009]         -2.6114      0.475     -5.493      0.000      -3.545      -1.678
C(year)[T.2010]         -1.9612      0.478     -4.100      0.000      -2.900      -1.022
C(year)[T.2011]         -0.2964      0.479     -0.619      0.536      -1.237       0.644
C(naics_code)[T.114]     6.2962      2.368      2.659      0.008       1.648      10.944
C(naics_code)[T.211]    -0.3049      2.416     -0.126      0.900      -5.047       4.437
C(naics_code)[T.212]    -2.6199      2.439     -1.074      0.283      -7.407       2.168
C(naics_code)[T.221]    -3.1735      1.705     -1.862      0.063      -6.520       0.173
C(naics_code)[T.236]     5.0526      2.281      2.215      0.027       0.575       9.531
C(naics_code)[T.237]     0.8639      3.559      0.243      0.808      -6.123       7.851
C(naics_code)[T.311]    -2.7032      2.386     -1.133      0.258      -7.388       1.981
C(naics_code)[T.312]     3.5415      1.513      2.341      0.019       0.572       6.511
C(naics_code)[T.313]    -3.5918      1.065     -3.372      0.001      -5.683      -1.501
C(naics_code)[T.314]    -1.8771      1.065     -1.762      0.078      -3.968       0.214
C(naics_code)[T.315]     4.7128      1.396      3.376      0.001       1.972       7.453
C(naics_code)[T.316]    -0.7542      1.550     -0.487      0.627      -3.797       2.289
C(naics_code)[T.321]    -2.9868      1.085     -2.752      0.006      -5.117      -0.856
C(naics_code)[T.322]    -6.5215      1.349     -4.834      0.000      -9.170      -3.873
C(naics_code)[T.323]    -4.7793      1.122     -4.260      0.000      -6.982      -2.577
C(naics_code)[T.324]    -3.0562      1.892     -1.615      0.107      -6.771       0.658
C(naics_code)[T.325]    -4.2280      1.732     -2.441      0.015      -7.628      -0.828
C(naics_code)[T.326]    -5.8176      1.307     -4.452      0.000      -8.383      -3.253
C(naics_code)[T.327]    -3.4417      1.273     -2.703    